# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Описание данных:
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
сalls — количество звонков,
minutes — суммарная длительность звонков в минутах,
messages — количество sms-сообщений,
mb_used — израсходованный интернет-трафик в Мб,
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# Проверим типы и выведем первые 5 строк
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# поменяем тип сообщений на целочисленный, так как сообщения не могут быть дробными
data['messages']=data['messages'].astype("int")

In [5]:
# проверим наличие дубликатов
data.duplicated().sum()

0

In [6]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Вывод:
Дубликатов не обнаружено
Пропущенных значений нет
Поменяли тип сообщений на целочисленный

## Разбейте данные на выборки

In [7]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra'] 

Так как спрятанной нет, то от основной для валидационной отделим 20%, 20% для тестовой, соотношение получается 3/1/1

(обучающая 60%, валидационная 20%, тестовая 20%)

In [8]:
# отделим 60% для обучающей выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=12345) 

In [9]:
# Разделим тестовую выборку пополам - на тестовую и валидационную
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, test_size=0.5, random_state=12345) 

## Исследуйте модели

Используем в качестве первой модели дерево решений и проверим наиболее эффективную глубину дерева

In [10]:
for depth in range(1,30):
    DT = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    DT.fit(features_train, target_train)
    predictions_valid = DT.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid)) 

max_depth = 1 : 0.7591785936527692
max_depth = 2 : 0.7846919726197884
max_depth = 3 : 0.7896701929060361
max_depth = 4 : 0.7865588052271313
max_depth = 5 : 0.790914747977598
max_depth = 6 : 0.8014934660858744
max_depth = 7 : 0.8027380211574362
max_depth = 8 : 0.8070939639079029
max_depth = 9 : 0.8046048537647791
max_depth = 10 : 0.8102053515868077
max_depth = 11 : 0.8083385189794648
max_depth = 12 : 0.8170504044803982
max_depth = 13 : 0.8189172370877411
max_depth = 14 : 0.8158058494088364
max_depth = 15 : 0.8133167392657125
max_depth = 16 : 0.8170504044803982
max_depth = 17 : 0.8139390168014935
max_depth = 18 : 0.8133167392657125
max_depth = 19 : 0.8077162414436839
max_depth = 20 : 0.8139390168014935
max_depth = 21 : 0.8126944617299315
max_depth = 22 : 0.8139390168014935
max_depth = 23 : 0.8139390168014935
max_depth = 24 : 0.8139390168014935
max_depth = 25 : 0.8139390168014935
max_depth = 26 : 0.8139390168014935
max_depth = 27 : 0.8139390168014935
max_depth = 28 : 0.8139390168014935
ma

С увеличением глубины результат улучшается и доходит до 0.81

In [11]:
best_DT = DecisionTreeClassifier(random_state=12345, max_depth=22) 
best_DT.fit(features_train, target_train)
predictions_valid = best_DT.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid)) 

0.8139390168014935


Используем в качестве модели случайный лес

In [12]:
best_model = None
best_result = 0
for est in range(1, 11):
    RF = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    RF.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = RF.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_RF = RF # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных
        
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.8500311138767891


Accuracy больше, чем 7.5 - 0.85

Проверим логистическую регрессию

In [13]:
LR = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 

LR.fit(features_train, target_train) 

LR.score(features_valid, target_valid) 

0.7554449284380834

Accuracy равен 0.76

Вывод - Лучший показатель accuracy у случайного леса - 0.85

## Проверьте модель на тестовой выборке

In [14]:
prediction_LR = LR.predict(features_test)
prediction_RF = best_RF.predict(features_test)
prediction_DT = best_DT.predict(features_test)

In [15]:
accuracy_LR = accuracy_score(prediction_LR, target_test)
accuracy_RF = accuracy_score(prediction_RF, target_test)
accuracy_DT = accuracy_score(prediction_DT, target_test)

In [16]:
print(f"Logistic Regression: {accuracy_LR}")
print(f"Random Forest: {accuracy_RF}")
print(f"Decision Tree: {accuracy_DT}")

Logistic Regression: 0.7373988799004356
Random Forest: 0.9738643434971997
Decision Tree: 1.0


Вывод:
Наиболее высокая accuracy вышла на дереве решений. 

## (бонус) Проверьте модели на адекватность

Наиболее адекватные предсказания делает Random Forest, это видно по точности предсказания на валидационной и на тестовой выборке

На валидационной выборке показала лучший результат модель случайный лес. Далее дерево решений и на последнем месте - логистическая регрессия. 
Удалось достичь Accuracy не меньше 0.75

Результаты на тестовой выборке: 

Logistic Regression: 0.7373988799004356

Random Forest: 0.9738643434971997

Decision Tree: 1.0

Результаты на валидационной выборке:

Logistic Regression: 0.7554449284380834

Random Forest: 0.8500311138767891

Decision Tree: 0.8139390168014935

